In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
train = pd.read_csv("data/train.csv")

In [6]:
train.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [7]:
train.shape

(5237980, 17)

# 特徴量
* stock_id - 銘柄のID。すべての銘柄のIDが存在するわけではない
* date_id - 日付ID。IDは連続したもので、全銘柄で一貫している。
* imbalance_size - 現在の基準価格での未マッチ額（単位：米ドル）。 
* imbalance_buy_sell_flag - オークションの不均衡の方向を反映する指標。 
    * 買い手側に偏りがある; 1
    * 売り手側に偏りがある; -1
    * 不均衡でない; 0
* reference_price - 対になる株式が最大になり、不均衡が最小になり、ビッドとアスクの中間点からの距離が最小になる価格。また、最良の買値と売値の間にあるニアプライスに等しいと考えることもできる。
* matched_size - 現在の基準価格でマッチング可能な金額（米ドル）。 
* far_price - オークションの関心のみに基づき、マッチング株数を最大化するクロス価格。この計算には連続成行注文は含まれない。 
* near_price - オークション注文と連続成行注文に基づくマッチング株数を最大化する交差価格。 
* [bid/ask]_price - ノン・オークション・ブックで最も競争力のある売買レベルの価格。 
* [bid/ask]_size - ノン・オークション・ブックにおける最も競争力のある売買レベルのドル想定元本。 
* wap - ノン・オークション・ブックにおける加重平均価格。 
    
* seconds_in_bucket - オークション開始からの経過秒数で、常に0から始まる。 
* target - その銘柄のワップにおける60秒後の将来の動きから、合成指数の60秒後の将来の動きを差し引いたもの。列車セットにのみ提供される。 
    * 合成指数は、このコンペティションのためにオプティバーが構築したナスダック上場銘柄のカスタム加重指数である。
    * ターゲットの単位はベーシス・ポイントで、これは金融市場で一般的な測定単位である。1ベーシスポイントの値動きは0.01%の値動きに相当する。
    * tは現在の観測時間であり、ターゲットを定義することができる：
* 各日付の最初のtime_idが0になった時（つまりseconds_in_bucketが0になった時）、APIは前の日付全体の真の目標値を提供するデータフレームを提供する。それ以外の行には、対象となるカラムのNULL値が含まれます。

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5237980 entries, 0 to 5237979
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   stock_id                 int64  
 1   date_id                  int64  
 2   seconds_in_bucket        int64  
 3   imbalance_size           float64
 4   imbalance_buy_sell_flag  int64  
 5   reference_price          float64
 6   matched_size             float64
 7   far_price                float64
 8   near_price               float64
 9   bid_price                float64
 10  bid_size                 float64
 11  ask_price                float64
 12  ask_size                 float64
 13  wap                      float64
 14  target                   float64
 15  time_id                  int64  
 16  row_id                   object 
dtypes: float64(11), int64(5), object(1)
memory usage: 679.4+ MB


In [12]:
train.isnull().sum()

stock_id                         0
date_id                          0
seconds_in_bucket                0
imbalance_size                 220
imbalance_buy_sell_flag          0
reference_price                220
matched_size                   220
far_price                  2894342
near_price                 2857180
bid_price                      220
bid_size                         0
ask_price                      220
ask_size                         0
wap                            220
target                          88
time_id                          0
row_id                           0
dtype: int64